In [1]:
import sys
sys.path.append('..')
from src.utils import helper
from src.utils.database import DatabaseConnection 

In [2]:
connection = DatabaseConnection(**helper.get_db_creds())

In [3]:
# What is the distribution of hostel ratings ?
query = """
    SELECT
    	star,
    	count(*) AS frequency
    FROM fact_hotel
    GROUP BY star
    ORDER BY star
"""

result = connection.read_sql(query)
print(result)

   star  frequency
0     1        154
1     2       1724
2     3       2382
3     4        676
4     5         90


In [4]:
# What is the rank hostels based on their stars within each city ?
query = """
    SELECT
    	di.name AS name,
    	da.city AS city,
    	fh.star,
    	DENSE_RANK() OVER (PARTITION BY da.city ORDER BY fh.star DESC) AS star_rank
    FROM fact_hotel AS fh
    JOIN dim_info AS di ON fh.info_id = di.id
    JOIN dim_adress AS da ON fh.adress_id = da.id;
"""

result = connection.read_sql(query)
print(result)

                               name                city  star  star_rank
0                 mercure abbeville           abbeville     4          1
1              hôtel ibis abbeville           abbeville     3          2
2            hôtel le relais vauban           abbeville     2          3
3          hôtel auberge du bel air       abreschviller     2          1
4     hôtel mercure golf cap d'agde                agde     4          1
...                             ...                 ...   ...        ...
5021        auberge du cheval blanc      yvoy-le-marron     3          1
5022                 hôtel gardenia               yvrac     3          1
5023             relais de la mothe  yzeures-sur-creuse     3          1
5024           hôtel le schlossberg          zellenberg     3          1
5025              hôtel du tourisme              zicavo     2          1

[5026 rows x 4 columns]


In [5]:
# Which region have the second most capacity ?
query = """
    WITH region AS (
    	SELECT
            	id,
            	LEFT(zip, 2) AS num
        	FROM dim_adress
    ),
    region_capacity AS (
    	SELECT
            	region.num AS region_num,
            	SUM(di.capacity) AS capacity_nb,
            	DENSE_RANK() OVER (ORDER BY SUM(di.capacity) DESC) AS capacity_rank
    	FROM fact_hotel AS fh
    	JOIN region ON fh.adress_id = region.id
    	JOIN dim_info AS di ON fh.info_id = di.id
    	GROUP BY region.num
    )
    SELECT
    	region_num,
    	capacity_nb
    FROM region_capacity
    WHERE capacity_rank = 2;
"""

result = connection.read_sql(query)
print(result)

  region_num  capacity_nb
0         92        19954


In [6]:
# Which region be a part of top 10 with most hotel who star rating above national average per region ?
query = """
    WITH region AS (
    	SELECT
            	id,
            	LEFT(zip, 2) AS num
        	FROM dim_adress
    ),
    avg_star AS (
    	SELECT
    		ROUND(AVG(star), 2) AS value
    	FROM fact_hotel
    ),
    hotel_data AS (
    	SELECT
    		region.num AS region_num,
    		COUNT(CASE WHEN fh.star > avg_star.value THEN 1 END) as hotel_count,
    		DENSE_RANK() OVER (ORDER BY COUNT(CASE WHEN fh.star > avg_star.value THEN 1 END) DESC) AS region_rank
    	FROM fact_hotel AS fh
    	JOIN region ON fh.adress_id = region.id,
    	avg_star
    	GROUP BY region.num
    )
    SELECT
    	region_num,
    	hotel_count,
    	region_rank
    FROM hotel_data
    WHERE region_rank <= 10;
"""

result = connection.read_sql(query)
print(result)

   region_num  hotel_count  region_rank
0          06          211            1
1          74          169            2
2          92           88            3
3          64           87            4
4          67           86            5
5          65           77            6
6          83           73            7
7          44           70            8
8          21           70            8
9          35           67            9
10         68           64           10
